In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
from matplotlib.image import NonUniformImage
import numpy as np
import scipy.integrate as integrate
import scipy.stats as lfit
from scipy.constants import pi, e, hbar, h
from scipy import interpolate, ndimage, signal, sparse, optimize, linalg, stats
from scipy.signal import savgol_filter
from ipywidgets import widgets, interact
from Delta_data import DatFile, Data2D
from functools import partial
from scipy.signal import find_peaks
from Delta_data_load import load_2d_data, get_gate1_lock1, get_gate3_lock1, get_grids
from Delta_peaks_hort import peaks_hort, find_peak_evol_hort, find_max_2D
import glob
print ("done")

done


In [2]:
plt.rcParams.update({'font.size': 16})

In [3]:
#Choose Excel file to read
df_runs = pd.read_excel("../VA_182MK_Sept2021/September-21.xlsx")
df_runs.shape



(620, 20)

In [4]:
#Convert empty "Step" cell to string.
#For convenience, I have deleted the step cells for any cancelled scan
#so the program can easily ignore cancelled scans
df_runs = df_runs[~df_runs["Step"].isnull()].astype({'Step': 'string'})
df_runs = df_runs[~df_runs["T"].isnull()].astype({'T': 'string'})
#df_runs = df_runs[df_runs["Other comments.1"].str.contains("2D")]

In [5]:
df_runs["T"].isnull().sum()

0

In [6]:
# Keep rows where the temperature is not null
df_runs = df_runs[~df_runs["Step"].isnull()].astype({'Step': 'string'})
# Keep rows where the temperature has a number
df_runs = df_runs[df_runs["Step"].str.contains('\d*\.\d+|\d+')]
df_runs = df_runs[df_runs["T"].str.contains('\d*\.\d+|\d+')]

In [7]:
col_name = 'T'
# storing cells that have mK/mk (not K) in m
m =  df_runs[col_name].str.contains('(\d*\.\d+|\d+)(mK|mk)')
# go to current T column and extract only the digits and 
# reassign it to the temp column

df_runs[col_name] = df_runs[col_name].str.extract('(\d*\.\d+|\d+)', expand=False).astype(float)
# convert the mK/mk into Kelvin 
df_runs.loc[m, col_name] *= 0.001

C:\Users\Madma\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [8]:
# convert mid gate column into string
df_runs["gate mid"] = df_runs["gate mid"].astype("string")
# extract only digits from mid gate column and convert to float
df_runs["gate mid"] = df_runs["gate mid"].str.extract('(\d*\.\d+|\d+)', expand=False).astype(float)

In [9]:
df_runs = df_runs[df_runs["Run"]<=409]
df_runs = df_runs[df_runs["Run"]>=355]
#df_runs = df_runs["Run"].astype("int")

In [10]:
'''Enter File Number to work work'''
'''Use only this file when entering delta values'''
file_number = 357



# Find the max value of the entire 2D scan OR of a specified square region
main_max, main_coord, file_data, data = find_max_2D(file_number, 
                                              global_max = True,     #If true, ignores values of x/y min/max and find global maximum
                                              x_min = 0.2625,
                                              x_max = 0.265,         #Enter dimensions of square you wish to find the maximum in
                                              y_min = 0.266,           #NUMBER COORESPONDS TO PIXEL DIMENSIONS
                                              y_max = 0.270,         #Bottom left pixel of 2D scan corresponds to (0,0)
                                              find_troughs=True)


print(main_max)
print(main_coord)
# print(file_data.z)
# print(file_data.z[2:4,2:4])
# newshit = file_data.z[2,2:4]
# print(newshit)

Max =  4.22437253e-06  located at pixel:  [5, 3] 
Gate 2V = 0.26049 
Gate 3V = 0.26763
4.22437253e-06
[5, 3]


In [11]:
file_list=[]
for ident in df_runs["Run"]:
    ident = int(ident)
    file_loc = glob.glob("../*/*Sept2021_{}.dat".format(ident), recursive=False)
    file_list.append(file_loc[0])

#print(file_list)
    
max_peaks = 2
fixedGate3Val = 0.265
x_array, y_array, width_array, prominence_array, num_peaks_array = \
                            find_peak_evol_hort(file_list=file_list, 
                                           fixedGate3Val=fixedGate3Val,    
                                           min_width=4, 
                                           height=1e-6,
                                           find_troughs=True,
                                           x_peak_max=0.5,
                                           x_peak_min=0.2,
                                           max_peaks=max_peaks,
                                           sort_prominence=True,    #True = sort peak data by prominence
                                           prominence=0.0000016)    #Average Prominenece for MKdata is 1E-6




#print(x_array)
#print(y_array)
#print(width_array)
#print(prominence_array)
# df_peaks = pd.DataFrame(x_array)
# df_peaks.to_csv("Data1015-1048/peaks.csv")


Finding peaks AND troughs
Ordering peaks by prominence
..\VA_182MK_Sept2021\VA_182MK_Sept2021_355.dat..\VA_182MK_Sept2021\VA_182MK_Sept2021_355.dat
..\VA_182MK_Sept2021\VA_182MK_Sept2021_357.dat..\VA_182MK_Sept2021\VA_182MK_Sept2021_357.dat
..\VA_182MK_Sept2021\VA_182MK_Sept2021_359.dat..\VA_182MK_Sept2021\VA_182MK_Sept2021_359.dat
..\VA_182MK_Sept2021\VA_182MK_Sept2021_363.dat..\VA_182MK_Sept2021\VA_182MK_Sept2021_363.dat
..\VA_182MK_Sept2021\VA_182MK_Sept2021_365.dat..\VA_182MK_Sept2021\VA_182MK_Sept2021_365.dat
..\VA_182MK_Sept2021\VA_182MK_Sept2021_367.dat..\VA_182MK_Sept2021\VA_182MK_Sept2021_367.dat
..\VA_182MK_Sept2021\VA_182MK_Sept2021_369.dat..\VA_182MK_Sept2021\VA_182MK_Sept2021_369.dat
..\VA_182MK_Sept2021\VA_182MK_Sept2021_371.dat..\VA_182MK_Sept2021\VA_182MK_Sept2021_371.dat
..\VA_182MK_Sept2021\VA_182MK_Sept2021_373.dat..\VA_182MK_Sept2021\VA_182MK_Sept2021_373.dat
..\VA_182MK_Sept2021\VA_182MK_Sept2021_375.dat..\VA_182MK_Sept2021\VA_182MK_Sept2021_375.dat
..\VA_182MK_Sep